<a href="https://colab.research.google.com/github/RandomForestRanger/PhD/blob/master/006_Messaround_with_Bernoulli_NB_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Pip installation
!pip install --upgrade -q gspread
!pip install gspread-dataframe
!pip install -U scikit-learn>=0.20
!pip install --upgrade scikit-learn

In [ ]:
#import libraries
import numpy as np
import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from sklearn.naive_bayes import ComplementNB

import sklearn.model_selection as ms
import sklearn.metrics as sklm

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import MultiLabelBinarizer # This one to later turn my LabelsNP into a onehot encoded 2d array

from sklearn import preprocessing
from sklearn.naive_bayes import BernoulliNB  # this will be operative today...
from sklearn.linear_model import LogisticRegression #Mb Added, not sure it should be here... 
import sklearn.model_selection as ms
import sklearn.metrics as sklm
import matplotlib.pyplot as plt
import numpy.random as nr

%matplotlib inline

In [ ]:
#authorise node to access Gdrive via KDL
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
# now containing "6 short munged and num only 30 Oct IRIS dataset"

#link to pre-opened csv file in my gdrive 
data1 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Q2YwmFiFSOkW_V9cRYgZ4GdGGl63_N6mOejEJQB1e0M/edit#gid=835063997')

In [ ]:
#get the data as a worksheet  
ws = data1.worksheet('icanhasdata')

In [ ]:
#convert the data to pandas dataframe (can include argument , header = None   if this is needed)
FullDF = get_as_dataframe(ws)
#inspect data
FullDF.tail()

In [ ]:
#since g-sheets brings along a lot of empty/Nan cellumns or rows, this chops them out. Can replace "all" with "any" if I want to discard any incomplete collumns or rows

# I changed the "all" to "any", to root out all cases of NaN...

FullDF.dropna(axis=1, how='all' ,inplace=True )
FullDF.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
FullDF.tail()

# if there are still "NaN" values, I want to fill them with "0". Go about this thusly:

FullDF.fillna(0)

In [ ]:
print(FullDF.shape)

In [ ]:
FeatDF  = FullDF.drop( columns = "PDV")
LabelDF = FullDF["PDV"]
print(FeatDF.shape)
print(LabelDF.shape)

In [ ]:
# Convent the DF to a NP array

FeatNP  = FeatDF.values
LabelNP = LabelDF.values
print("Feat:")
print(FeatNP.shape)
print(FeatNP)
print("Label:")
print(LabelNP.shape)

In [ ]:
print(FeatNP[0:1,0:])

In [ ]:
## Randomly sample cases to create independent training and test data
nr.seed(1)
indx = range(FeatNP.shape[0])
indx = ms.train_test_split(indx, test_size = 100)
X_train = FeatNP[indx[0],:]
y_train = np.ravel(LabelNP[indx[0]])
X_test = FeatNP[indx[1],:]
y_test = np.ravel(LabelNP[indx[1]])

In [ ]:
print("X train shape is: ", X_train.shape, "\n")
print("X test shape is: ", y_test.shape, "\n")
print("Y train shape is: ", y_train.shape, "\n")
print("Y test shape is: ", y_test.shape, "\n")
print("Unscaled X train record number 700 looks like this: \n", X_train[700:701,:])
print(np.isnan(X_train.any()))
print(np.isfinite(X_train.any()))

In [ ]:
## ONLY scale if I'm going to Gaussian...
#scale = preprocessing.StandardScaler()
#scale.fit(X_train)
#X_train = scale.transform(X_train)

In [ ]:
#print("Scaled X train record number 700 looks like this: \n", X_train[700:701,:])

In [ ]:
NB_mod = BernoulliNB()

##from sklearn.naive_bayes import MultinomialNB
##NB_mod = MultinomialNB()
##MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
###NB_mod = ComplementNB()

NB_mod.fit(X_train, y_train)
BernoulliNB(alpha=1.0, class_prior=None, fit_prior=True, )

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [ ]:
#   X_test = scale.transform(X_test)
scores = NB_mod.predict(X_test)

In [ ]:
## What, this worked ?!!   ##
# MB trying to figure out how to see the coefficients/parameters...
print(NB_mod.coef_)
 

In [ ]:
y_test

array(['V', 'NP', 'V', 'P', 'NP', 'P', 'D', 'D', 'D', 'V', 'P', 'NP', 'P',
       'P', 'P', 'P', 'NP', 'D', 'P', 'NP', 'D', 'P', 'NP', 'NP', 'V',
       'P', 'NP', 'D', 'P', 'NP', 'P', 'NP', 'NP', 'NP', 'NP', 'P', 'P',
       'P', 'NP', 'NP', 'NP', 'NP', 'V', 'NP', 'NP', 'P', 'P', 'NP', 'NP',
       'D', 'P', 'NP', 'NP', 'P', 'NP', 'D', 'NP', 'D', 'NP', 'NP', 'P',
       'NP', 'NP', 'P', 'V', 'NP', 'P', 'NP', 'P', 'NP', 'P', 'D', 'P',
       'P', 'NP', 'V', 'NP', 'NP', 'D', 'NP', 'D', 'P', 'V', 'NP', 'NP',
       'P', 'P', 'P', 'P', 'P', 'NP', 'P', 'NP', 'P', 'NP', 'P', 'NP',
       'NP', 'NP', 'D'], dtype=object)

In [ ]:
scores

array(['D', 'NP', 'P', 'P', 'NP', 'NP', 'D', 'NP', 'P', 'V', 'NP', 'NP',
       'NP', 'P', 'P', 'NP', 'NP', 'NP', 'P', 'P', 'V', 'P', 'NP', 'NP',
       'P', 'P', 'P', 'NP', 'P', 'NP', 'P', 'P', 'NP', 'NP', 'NP', 'P',
       'NP', 'P', 'NP', 'NP', 'NP', 'NP', 'V', 'NP', 'NP', 'P', 'NP',
       'NP', 'NP', 'D', 'P', 'NP', 'NP', 'P', 'NP', 'P', 'NP', 'P', 'NP',
       'NP', 'D', 'V', 'NP', 'P', 'D', 'NP', 'D', 'P', 'P', 'NP', 'V',
       'NP', 'P', 'NP', 'P', 'V', 'NP', 'NP', 'V', 'NP', 'D', 'P', 'V',
       'NP', 'D', 'P', 'NP', 'NP', 'P', 'P', 'NP', 'P', 'NP', 'NP', 'V',
       'P', 'P', 'NP', 'NP', 'P'], dtype='<U2')

In [ ]:
def print_metrics_4(labels, scores):
   
    conf = sklm.confusion_matrix(labels, scores)
    print('                 Confusion matrix')
    print('                 Score NP         Score P          Score D          Score V')
    print('Actual NP      %6d' % conf[0,0] + '            %5d' % conf[0,1] + '             %5d' % conf[0,2]+ '             %5d' % conf[0,3])
    print('Actual P       %6d' % conf[1,0] + '            %5d' % conf[1,1] + '             %5d' % conf[1,2]+ '             %5d' % conf[1,3])
    print('Actual D       %6d' % conf[2,0] + '            %5d' % conf[2,1] + '             %5d' % conf[2,2]+ '             %5d' % conf[2,3])
    print('Actual V       %6d' % conf[3,0] + '            %5d' % conf[3,1] + '             %5d' % conf[3,2]+ '             %5d' % conf[3,3])
    ## Now compute and display the accuracy and metrics
    print('')
    print('Accuracy        %0.2f' % sklm.accuracy_score(labels, scores))
    metrics = sklm.precision_recall_fscore_support(labels, scores)
    print(' ')
    print('            NP        P          D          V')
    print('Num case    %0.2f' % metrics[3][0] + '     %0.2f' % metrics[3][1] + '      %0.2f' % metrics[3][2]+ '      %0.2f' % metrics[3][3])
    print('Precision   %0.2f' % metrics[0][0] + '      %0.2f' % metrics[0][1] + '       %0.2f' % metrics[0][2]+ '       %0.2f' % metrics[0][3])
    print('Recall      %0.2f' % metrics[1][0] + '      %0.2f' % metrics[1][1] + '       %0.2f' % metrics[1][2]+ '       %0.2f' % metrics[1][3])
    print('F1          %0.2f' % metrics[2][0] + '      %0.2f' % metrics[2][1] + '       %0.2f' % metrics[2][2]+ '       %0.2f' % metrics[2][3])
    
print_metrics_4(y_test, scores) 

# with 110 variables, GaussianNB   accuracy is 0.12
# with 89 variables,  GaussianNB   accuracy is 0.20 
# with 89 variables,  ComplementNB accuracy is 0.33
# with 49 variables,  ComplementNB accuracy is 0.55
# when i cut out the Unique ID, accuracy moved up to 0.58
# Using Bernoulli, I move this up to 0.64 ...(should I cross-validate?)  ...should I run this more than once?

                 Confusion matrix
                 Score NP         Score P          Score D          Score V
Actual NP           3                4                 4                 2
Actual P            1               35                 6                 2
Actual D            2               10                22                 1
Actual V            2                0                 2                 4

Accuracy        0.64
 
            NP        P          D          V
Num case    13.00     44.00      35.00      8.00
Precision   0.38      0.71       0.65       0.44
Recall      0.23      0.80       0.63       0.50
F1          0.29      0.75       0.64       0.47


## Cross validate model

To compute a better estimate of model performance, you can perform simple cross validation. The code in the cell performs the following processing:
1. Create a list of the metrics to be computed for each fold. 
2. Defines a logistic regression model object.
3. A 10 fold cross validation is performed using the `cross_validate` function from the scikit-learn `model_selection` package.

Execute this code. 

In [ ]:
# First I must binarize my labels
MultiLabelBinarizer().fit_transform(LabelNP)



array([[0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       ...,
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 1, 1, 0]])

In [ ]:
from sklearn.model_selection import RepeatedKFold

In [ ]:
a = ms.RepeatedKFold(n_splits = 5, n_repeats = 10, random_state = 123)

In [ ]:
#LabelNP = LabelNP.reshape(LabelNP.shape[0],)
scoring = ['precision_macro', 'recall_macro', 'roc_auc']
#logistic_mod = linear_model.LogisticRegression(C = 1.0, class_weight = {0:0.45, 1:0.55}) 
scores = ms.cross_validate(NB_mod, FeatNP, LabelNP, scoring=scoring,
                        cv=5, return_train_score = True)

ValueError: ignored

The code in the cell below displays the performance metrics along with the mean and standard deviation, computed for each fold to the cross validation. The 'macro' versions of precision and recall are used. These macro versions average over the positive and negative cases. 

Execute this code, examine the result, and answer **Question 1** on the course page.

In [ ]:
def print_format(f,x,y,z):
    print('Fold %2d    %4.3f        %4.3f      %4.3f' % (f, x, y, z))

def print_cv(scores):
    fold = [x + 1 for x in range(len(scores['test_precision_macro']))]
    print('         Precision     Recall       AUC')
    [print_format(f,x,y,z) for f,x,y,z in zip(fold, scores['test_precision_macro'], 
                                          scores['test_recall_macro'],
                                          scores['test_roc_auc'])]
    print('-' * 40)
    print('Mean       %4.3f        %4.3f      %4.3f' % 
          (np.mean(scores['test_precision_macro']), np.mean(scores['test_recall_macro']), np.mean(scores['test_roc_auc'])))  
    print('Std        %4.3f        %4.3f      %4.3f' % 
          (np.std(scores['test_precision_macro']), np.std(scores['test_recall_macro']), np.std(scores['test_roc_auc'])))

print_cv(a)    

TypeError: ignored

Notice that there is considerable variability in each of the performance metrics from fold to fold. Even so, the standard deviations are at least an order of magnitude than the means. It is clear that **any one fold does not provide a representative value of the performance metrics**. The later is a key point as to why cross validation is important when evaluating a machine learning model.  

Compare the performance metric values to the values obtained for the baseline model you created above. In general the metrics obtained by cross validation are lower. However, the metrics obtained for the baseline model are within 1 standard deviation of the average metrics from cross validation. 

## Optimize hyperparameters with nested cross validation

Given the variability observed in cross validation, it should be clear that performing model selection from a single training and evaluation is an uncertain proposition at best. Fortunately, the nested cross validation approach provides a better way to perform model selection. However, there is no guarantee that a model selection process will, in fact, improve a model. In some cases, it may prove to be that model selection has minimal impact. 

To start the nested cross validation process it is necessary to define the randomly sampled folds for the inner and outer loops. The code in the cell below uses the `KFolds` function from the scikit-learn `model_selection` package to define fold selection objects. Notice that the `shuffle = True` argument is used in both cases. This argument specifies that a random shuffle is preformed before folds are created, ensuring that the sampling of the folds for the inside and outside loops are independent. Notice that by creating these independent fold objects there is no need to actually create nested loops for this process. 

Execute this code.

In [ ]:
nr.seed(123)
inside = ms.KFold(n_splits=10, shuffle = True)
nr.seed(321)
outside = ms.KFold(n_splits=10, shuffle = True)

An important decision in model selection searches is the choice of performance metric used to find the best model. For classification problems scikit-learn uses accuracy as the default metric. However, as you have seen previously, accuracy is not necessarily the best metric, particularly when there is a class imbalance as is the case here. There are a number of alternatives which one could choose for such a situation. In this case AUC will be used. 

The code below uses the `inside` k-fold object to execute the inside loop of the nested cross validation. Specifically, the steps are:
1. Define a dictionary with the grid of parameter values to search over. In this case there is only one parameter, `C`, with a list of values to try. In a more general case, the dictionary can contain values from multiple parameters, creating a multi-dimensional grid that the cross validation process will iterate over. In this case there are 5 hyperparameter values in the grid and 10-fold cross validation is being used. Thus, the model will be trained and evaluated 50 times. 
2. The logistic regression model object is defined. 
3. The cross validation search over the parameter grid is performed using the `GridSearch` function from the scikit-learn `model_selection` package. Notice that the cross validation folds are computed using the `inside` k-fold object.


****
**Note:** Somewhat confusingly, the scikit-learn `LogisticRegression` function uses a regularization parameter `C` which is the inverse of the usual l2 regularization parameter $\lambda$. Thus, the smaller the parameter the stronger the regulation 
****

Execute this code.

In [ ]:
nr.seed(3456)
## Define the dictionary for the grid search and the model object to search on
param_grid = {"C": [0.1, 1, 10, 100, 1000]}
## Define the logistic regression model
logistic_mod = linear_model.LogisticRegression(class_weight = {0:0.45, 0:0.55}) 

## Perform the grid search over the parameters
clf = ms.GridSearchCV(estimator = logistic_mod, param_grid = param_grid, 
                      cv = inside, # Use the inside folds
                      scoring = 'roc_auc',
                      return_train_score = True)

The cross validated grid search object, `clf`, has been created. 

The code in the cell below fits the cross validated model using the `fit`method. The AUC for each hyperparameter and fold is displayed as an array. Finally, the hyperparameter for the model with the best average AUC is displayed.  Execute this code and  examine the results.

In [ ]:
## Fit thhe cross validated grid search over the data 
clf.fit(Features, Labels)
keys = list(clf.cv_results_.keys())
for key in keys[6:16]:
    print(clf.cv_results_[key])
## And print the best parameter value
clf.best_estimator_.C

The array of AUC metrics has dimensions 10 folds X  hyperparameter values. As you might expect by now, there is considerable variation in the AUC from fold to fold for each hyperparamter value, or column. 

Evidently, the optimal hyperparameter value is 0.1. 

To help understand this behavior a bit more, the code in the cell below does the following:
1. Compute and display the mean and standard deviation of the AUC for each hyperparameter value.
2. Plot the AUC values for each fold vs. the hyperparameter values. The mean AUC for each hyperparameter value is shown with a red +. 

Execute this code and examine the results. 

In [ ]:
def plot_cv(clf, params_grid, param = 'C'):
    params = [x for x in params_grid[param]]
  
    keys = list(clf.cv_results_.keys())              
    grid = np.array([clf.cv_results_[key] for key in keys[6:16]])
    means = np.mean(grid, axis = 0)
    stds = np.std(grid, axis = 0)
    print('Performance metrics by parameter')
    print('Parameter   Mean performance   STD performance')
    for x,y,z in zip(params, means, stds):
        print('%8.2f        %6.5f            %6.5f' % (x,y,z))
    
    params = [math.log10(x) for x in params]
    
    plt.scatter(params * grid.shape[0], grid.flatten())
    p = plt.scatter(params, means, color = 'red', marker = '+', s = 300)
    plt.plot(params, np.transpose(grid))
    plt.title('Performance metric vs. log parameter value\n from cross validation')
    plt.xlabel('Log hyperparameter value')
    plt.ylabel('Performance metric')
    
plot_cv(clf, param_grid)    

There are a number of points to notice here:
1. The mean AUC for each value of the hyperparameter are all within 1 standard deviation of each other. This result indicates that model performance is not sensitive to the choice of hyperparamter. 
2. Graphically you can see that there is a noticeable variation in the AUC from metric to metric, regardless of hyperparameter. Keep in mind that **this variation is simply a result of random sampling of the data!**

Finally, it is time to try execute the outer loop of the nested cross validation to evaluate the performance of the 'best' model selected by the inner loop. In this case, 'best' is quite approximate, since as already noted, the differences in performance between the models is not significant. 

The code in the cell below executes the outer loop of the nested cross validation using the `cross_val_scores` function from the scikit-learn `model_selection` package. The folds are determined by the `outside` k-fold object. The mean and standard deviation of the AUC is printed along with the value estimated for each fold. Execute this code and examine the result. 

Then, answer **Question 2** on the course page.

In [ ]:
nr.seed(498)
cv_estimate = ms.cross_val_score(clf, Features, Labels, 
                                 cv = outside) # Use the outside folds
print('Mean performance metric = %4.3f' % np.mean(cv_estimate))

print('SDT of the metric       = %4.3f' % np.std(cv_estimate))
print('Outcomes by cv fold')
for i, x in enumerate(cv_estimate):
    print('Fold %2d    %4.3f' % (i+1, x))

As expected, there is considerable variation in AUC across the folds. The mean AUC is a bit lower than estimated for the inner loop of the nested cross validation and the baseline model. However, all of these values are within 1 standard deviation of each other, and thus these differences cannot be considered significant. 

Now, you will build and test a model using the estimated optimal hyperparameters. Then, answer **Question 3** on the course page. 

In [ ]:
logistic_mod = linear_model.LogisticRegression(C = 0.1, class_weight = {0:0.45, 1:0.55}) 
logistic_mod.fit(X_train, y_train)
probabilities = logistic_mod.predict_proba(X_test)
print_metrics(y_test, probabilities, 0.3)  
plot_auc(y_test, probabilities)

## Summary

In this lab you have performed by simple cross validation and nested cross validation. Key points and observations are:
1. Model selection should be done using a resampling procedure such as nested cross validation. The nested sampling structure is required to prevent bias in model selection wherein the model selected learns the best hyperparameters for the samples used, rather than a model that generalizes well. 
2. There is significant variation in model performance from fold to fold in cross validation. This variation arises from the sampling of the data alone and is not a property of any particular model.
3. Given the expected sampling variation in cross validation, there is generally considerable uncertainty as to which model is best when performing model selection.  